In [1]:
import numpy as np
import sys  # 导入sys模块
import time
sys.setrecursionlimit(3000)

# 录入table数据

In [2]:
def creat_new_table():
    # 每一纬度依次为 num_stage,group，row，col, 可能、不可能、确定 依次为 1，0，10
    table=np.ones((9,9,9,9)).astype(np.int32)
    # 对与grop 进行定义
    group_list=[
        1,1,1,2,2,2,3,3,3,
        1,1,1,2,2,2,3,3,3,
        1,1,1,2,2,2,3,3,3,
        4,4,4,5,5,5,6,6,6,
        4,4,4,5,5,5,6,6,6,
        4,4,4,5,5,5,6,6,6,
        7,7,7,8,8,8,9,9,9,
        7,7,7,8,8,8,9,9,9,  
        7,7,7,8,8,8,9,9,9,  
    ]

    group_np=np.array(group_list).reshape(9,9)

    #每一个num_stage
    for num_stage in range(0,9):

        #每一个group
        for group in range(0,9):

            table[num_stage,group]=np.where(group_np!=group+1,0,table[num_stage,group])
            
    return table

In [3]:
# 知道row,col,求group
def get_group(row,col):
    
    group_list=[
        1,1,1,2,2,2,3,3,3,
        1,1,1,2,2,2,3,3,3,
        1,1,1,2,2,2,3,3,3,
        4,4,4,5,5,5,6,6,6,
        4,4,4,5,5,5,6,6,6,
        4,4,4,5,5,5,6,6,6,
        7,7,7,8,8,8,9,9,9,
        7,7,7,8,8,8,9,9,9,  
        7,7,7,8,8,8,9,9,9,  
    ]

    group_np=np.array(group_list).reshape(9,9)
    group=group_np[row,col]-1
    return group
    

In [4]:
#根据提供的数据更新table
def input_table(start_np):
    table=creat_new_table()
    the_index=np.argwhere(start_np!=0)
    for index in the_index:
        row,col=index
        num_stage=start_np[row,col]-1
        table=rebuilt_table_by_blog(table,num_stage,row,col)
        
    return table

# 更新table

### 根据已经确定的num_stage 进行更新

In [5]:
# table_sum 里有1 将这块改为10
def sum_1_to_10(table):
    # 如果一个blog只有1种num_stage 就替换为10
    table_sum=np.sum(table,axis=(0,1))
    
   
    the_index=np.argwhere(table_sum==1)
    for index in the_index:
        row,col=index
        group=get_group(row,col)
        num_stage=get_blog_numstage(table,1,group,row,col)
        if num_stage:
            table=rebuilt_table_by_blog(table,num_stage[0],row,col)

    return table

In [6]:
# 已确定blog的num_stage，更新相关数据
def rebuilt_table_by_blog(table,num_stage,row,col):
    #将同一group的更新0
    group=get_group(row,col)
    
#     print('更新row%d,col%d,group%d 为%d'%(row,col,group,num_stage+1))
    table[num_stage,group,:,:]=0
    table[num_stage,:,:,col]=0
    table[num_stage,:,row,:]=0
    table[:,:,row,col]=0
    table[num_stage,group,row,col]=10
    
    return table

In [7]:

#根据已有信息重塑table
def rebuilt_table(table):
    
    while (np.sum(table,axis=(0,1))==1).any():
        table=sum_1_to_10(table)
        
    return table

In [8]:
def rebuilt_table_twin(table):
    while ~(table==check_twin(table)).all():
        table=check_twin(table)
        table=rebuilt_table(table)
    return table
        

### 对于twin_blog 进行更新

In [9]:
# 取得每个数的super_num
def get_SuperNum(table,row,col):
    plus_np=[100000000,20000000,3000000,
             400000,50000,6000,
             700,80,9]
    plus_np=np.array(plus_np)
    min_num=np.sum(table,axis=1)[:,row,col]
    return sum(plus_np*min_num)

In [10]:
# 取得blog 为1的数位
def get_numstage_list(table,row,col,num):
    stage_list=np.argwhere(np.sum(table[:,:,row,col],axis=1)==num)
    return stage_list

In [11]:
# 获得check_np
def get_check_np(table):
    # 取得还有两种可能性的数
    twin_index=np.argwhere(np.sum(table,axis=(0,1))==2)
    superlist=[]
    grouplist=[]
    for index in twin_index:
        row,col=index
        grouplist.append(get_group(row,col))    
        superlist.append(get_SuperNum(table,row,col))

    check_np=np.column_stack((np.array(grouplist),twin_index,np.array(superlist)))
   
    
    return check_np
    

In [12]:
def check_group_row_col(table,check_np,minus_np,check_row,add_row):
    group,row,col=check_np[check_row][0:3]
    second_group,second_row,second_col=check_np[check_row+add_row+1][0:3]
    the_numstage_np=get_numstage_list(table,row,col,1)

    #如果同group
    if minus_np[add_row][0]==0:
        table[:,group,:,:][the_numstage_np]=0       

    elif minus_np[add_row][1]==0:
        table[:,:,row,:][the_numstage_np]=0  


    elif minus_np[add_row][2]==0:
        table[:,:,:,col][the_numstage_np]=0
    
    


    table[:,group,row,col][the_numstage_np]=1
    table[:,second_group,second_row,second_col][the_numstage_np]=1
    
    return table
        

In [13]:
def check_minus(table,check_np,minus_np,check_row):
    minus_np.shape[0]
    add_row=0
    while add_row<minus_np.shape[0]:
        if (minus_np[add_row][3]==0)&(0 in minus_np[add_row][0:3]):
            #检查是否有group，row,col
            table=check_group_row_col(table,check_np,minus_np,check_row,add_row)

        add_row+=1
    return table


In [14]:
# 检查table里的twin
def check_twin(table):
    check_np=get_check_np(table)
    
    n=check_np.shape[0]
    check_row=0
    while check_row<n:
        minus_np=check_np[check_row+1:n]-check_np[check_row]
        table=check_minus(table,check_np,minus_np,check_row)
        check_row+=1
    return table
    

## 判断计算结果

In [15]:
def get_blog_numstage(table,num_tag,group,row,col):
    
    return np.argwhere(table[:,group,row,col]==num_tag).T[0].tolist()

In [16]:
# 判断是否求出正确解，用于递归
def recheck_answer(table):
    # 校核是否满足要求
    # 每个位置是否只有一个确定
    if all(
        [
            (np.sum(table,axis=(0,1))==10).all(),
           (np.sum(table,axis=(1,2))==10).all(),
           (np.sum(table,axis=(1,3))==10).all(),
           (np.sum(table,axis=(2,3))==10).all()
        ]
    ):
        print('yes ,fond it!!!')
        return True
    else:
        print('sorry ,not it!!!')
        return False
        
  
    


In [17]:
# 判断是否求出正确解
def recheck_table(table):
    # 校核是否满足要求
    # 每个位置是否只有一个确定
    if all(
        [
            (np.sum(table,axis=(1,2))==0).any(),
         (np.sum(table,axis=(1,3))==0).any(),
         (np.sum(table,axis=(2,3))==0).any()

        ]
    ):
#         print('Warning ,it is not !!!')
        return False
    else:
#         print('yeah ,go on!!!')
        return True

## 递归求answer

In [18]:
# 使用递归来尝试找到正确答案

def test_answer(table):
#     print('_'*20)
    #如果np.sum 都为10
    if (np.sum(table,axis=(0,1))==10).all():
        
        answer.append(table)
        
    else:
        
        #找到table里只有2个可能性的，给一个赋值        
        table_sum=np.sum(table,axis=(0,1))
        the_2_index=np.argwhere(table_sum==2)
        
        if the_2_index.any():
            
            row,col=the_2_index[0]
            group=get_group(row,col)
            num_list=get_blog_numstage(table,1,group,row,col)
            for num_stage in num_list:
                new_table=table.copy()
                new_table=rebuilt_table_by_blog(new_table,num_stage,row,col)
                if recheck_table(new_table):
                    new_table=rebuilt_table(new_table)
                    
                    test_answer(new_table)

## 展示计算结果

In [19]:
def get_bignum(table,index):
    row,col=index

    num_list=np.sum(table[:,:,row,col],axis=1)

    numstage_list=np.argwhere(num_list==1)
    numstage_list=(numstage_list+1)[:,0].tolist()
    
    big_num=0
    plush_num=1
    i=len(numstage_list)-1
    while i>-1:
        
        big_num+=numstage_list[i]*plush_num
        plush_num*=10
        i-=1
    
    return big_num
    

In [20]:
# show 解数进展
def show_table(table):
    the_index=np.argwhere(table==10)    
    show_table=np.zeros((9,9)).astype(np.int32)
    for index in the_index:
        num_stage,group,row,col=index
        show_table[row,col]=num_stage+1
    #未确定num—stage的数
    table_sum=np.sum(table,axis=(0,1))
    the_index=np.argwhere(table_sum!=10)
    for index in the_index:
        big_num=get_bignum(table,index)
        row,col=index
        show_table[row,col]=big_num
        
    return show_table


In [21]:
# 展示计算结果
def show_answer(answer):
    print('数独结果是：')
    print('\n')
    for i in answer:
        print('_'*20)
        print('\n')
        print(show_table(i))
    
        

# 测试

## 录入初始信息

In [22]:
# 开局信息1
start_list1=[
    0,8,0,0,0,7,0,0,6,
    0,3,9,2,0,8,0,0,0,
    4,0,0,1,3,0,0,0,0,
    9,0,5,0,0,0,0,0,4,
    0,0,3,0,0,0,0,0,0,
    6,2,0,0,0,5,0,0,0,
    8,0,1,0,0,6,2,0,0,
    0,0,0,5,0,0,0,0,0,
    0,6,0,3,0,0,8,0,0
]


In [23]:
# 开局信息_2
start_list2=[
    5,0,2,0,0,0,0,8,0,
    0,0,1,0,0,0,0,0,0,
    0,0,0,4,0,3,0,0,0,
    0,4,0,0,0,0,0,0,0,
    0,0,0,0,0,6,7,0,9,
    0,0,8,0,5,0,0,0,0,
    0,0,0,0,2,1,0,0,0,
    0,9,0,0,0,0,0,0,3,
    0,7,0,0,0,0,6,0,0
]


In [24]:
# 开局信息_3
start_list3=[
    0,0,0,0,5,8,0,0,0,
    7,6,0,0,0,2,3,0,4,
    0,0,0,3,4,0,8,2,0,
    0,7,0,0,0,0,0,0,1,
    0,0,0,0,0,0,6,4,0,
    1,3,2,4,6,0,9,0,0,
    3,0,0,7,9,0,2,0,5,
    0,0,7,0,0,4,1,0,0,
    5,9,0,0,0,6,0,0,8,
]

In [25]:
# 开局信息_4
start_list4=[
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
    0,0,0,0,0,0,0,0,0,
  ]


In [26]:
# 开局信息_4
start_list4=[
    0,4,6,9,0,3,0,0,0,
    0,0,3,0,5,0,0,6,0,
    9,0,0,0,0,2,0,0,3,
    0,0,5,0,0,6,0,0,0,
    8,0,0,0,0,0,0,1,0,
    0,1,0,7,8,0,2,0,0,
    0,0,0,0,0,0,0,5,0,
    0,8,1,3,0,0,0,0,7,
    0,0,0,8,0,0,1,0,4,
  ]


In [27]:
#计算开始时间
time_start=time.time()
#转为 numpy格式
start_np=np.array(start_list4).reshape((9,9))
# 信息更新进table
table=input_table(start_np)

In [28]:
table=rebuilt_table(table)
table=rebuilt_table_twin(table)

In [33]:
#展示过程量
print(show_table(table))

[[  1257      4      6      9     17      3    578    278   1258]
 [   127     27      3     14      5   1478   4789      6   1289]
 [     9     57     78    146   1467      2   4578    478      3]
 [  2347   2379      5    124  12349      6  34789  34789     89]
 [     8  23679   2479    245   2349    459 345679      1    569]
 [   346      1     49      7      8    459      2    349    569]
 [ 23467  23679   2479   1246 124679   1479   3689      5   2689]
 [  2456      8      1      3   2469    459     69     29      7]
 [ 23567 235679    279      8   2679    579      1    239      4]]


In [30]:
# 进行递归求解
answer=[]
test_answer(table)

In [31]:
time_end=time.time()

In [35]:
show_answer(answer)
print('\n')
print('计算耗时：%.2f s'%(time_end-time_start))

数独结果是：


____________________


[[1 4 6 9 7 3 5 8 2]
 [7 2 3 4 5 8 9 6 1]
 [9 5 8 6 1 2 4 7 3]
 [3 7 5 1 2 6 8 4 9]
 [8 9 2 5 3 4 7 1 6]
 [6 1 4 7 8 9 2 3 5]
 [4 6 7 2 9 1 3 5 8]
 [2 8 1 3 4 5 6 9 7]
 [5 3 9 8 6 7 1 2 4]]


计算耗时：0.10 s
